<a href="https://colab.research.google.com/github/viggopatel/QM2-real/blob/main/GDPpCap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install Basemap
!pip install ipyleaflet

In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pylab
from datetime import datetime

%matplotlib inline
pylab.rcParams['figure.figsize'] = (10, 8)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
df = pd.read_csv('/content/drive/MyDrive/gdp_pcap.csv')

In [14]:
df=df.rename(columns={'Latitutde': 'Latitude'})
df

,country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Latitude,Longitude
0,Algeria,11k,11.1k,11.2k,11.2k,11.4k,11.6k,11.8k,11.7k,11.7k,11.6k,10.8k,28.0339° N,1.6596° E
1,Angola,7690,7660,8010,8100,8180,7970,7490,7220,6880,6600,6030,11.2027° S,17.8739° E
2,Bangladesh,3400,3570,3760,3930,4120,4340,4590,4830,5120,5470,5590,24°N,90°E
3,Benin,2630,2630,2680,2790,2880,2850,2860,2930,3040,3160,3190,9.3077° N,2.3158° E
4,Brazil,14.9k,15.3k,15.5k,15.8k,15.7k,15k,14.4k,14.5k,14.7k,14.7k,14.1k,14.235° S,51.9253° W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
!pip install geopy==2.3.0

from geopy.geocoders import Nominatim

In [16]:
from ipyleaflet import Map, CircleMarker, TileLayer
import re  # Import the re module
import pandas as pd
# Assuming df_2010 is your DataFrame with 'Latitude', 'Longitude', and 'Value' columns
# Filter out rows with missing coordinates to avoid errors in map creation

def clean_coordinate(coord):
    try:
        coord = str(coord)
        # Extract direction (N, S, E, W)
        direction = re.search(r"[NSEW]", coord)
        if direction:
            direction = direction.group(0)
        else:
            direction = None

        # Extract numeric part
        coord = re.sub(r"[°NSEW]", "", coord)
        coord = float(coord)
        # Apply direction to get decimal degrees
        if direction in ('S', 'W'):
            coord *= -1  # Make negative for South or West
        return coord
    except (TypeError, ValueError):  # Catch potential errors
        return 0  # Return 0 as a default value

# Apply the function to Latitude and Longitude columns
df['Latitude'] = df['Latitude'].apply(clean_coordinate)
df['Longitude'] = df['Longitude'].apply(clean_coordinate)

df = df.dropna(subset=['Latitude', 'Longitude'])
center = (df['Latitude'].mean(), df['Longitude'].mean())

m = Map(center=center, zoom=2)

world_map = TileLayer(
    url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
    attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
)
m.add_layer(world_map)

for index, row in df.iterrows():
    marker = CircleMarker(
        location=(row['Latitude'], row['Longitude']),
        radius=5,  # You might need to adjust the radius based on your data
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
    )
    m.add_layer(marker)

display(m)

Map(center=[8.130765306122449, 15.426828571428572], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [17]:
# prompt: if the value in columns 1:11 contains "k", remove k and multiply the number by 1,000

import pandas as pd
import re

# ... (Your existing code)

def clean_and_convert(value):
    if isinstance(value, str) and 'k' in value:
        try:
            numeric_part = float(value.replace('k', ''))
            return numeric_part * 1000
        except ValueError:
            return value  # Return original value if conversion fails
    return value

for col in df.columns[0:11]:  # Apply to columns 1 to 11 (index 0 to 10)
    df[col] = df[col].apply(clean_and_convert)

df

<ipython-input-17-c7091a554057>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(clean_and_convert)


,country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Latitude,Longitude
0,Algeria,11000.0,11100.0,11200.0,11200.0,11400.0,11600.0,11800.0,11700.0,11700.0,11600.0,10.8k,28.0339,1.6596
1,Angola,7690,7660,8010,8100,8180,7970,7490,7220,6880,6600,6030,-11.2027,17.8739
2,Bangladesh,3400,3570,3760,3930,4120,4340,4590,4830,5120,5470,5590,24.0000,90.0000
3,Benin,2630,2630,2680,2790,2880,2850,2860,2930,3040,3160,3190,9.3077,2.3158
4,Brazil,14900.0,15300.0,15500.0,15800.0,15700.0,15000.0,14400.0,14500.0,14700.0,14700.0,14.1k,-14.2350,-51.9253
5,Cameroon,3310,3320,3380,3450,3540,3630,3680,3700,3740,3770,3680,7.3697,12.3547
6,China,8880,9680,10400.0,11100.0,11900.0,12600.0,13400.0,14200.0,15100.0,16000.0,16.3k,35.8617,104.1954
7,Colombia,11900.0,12600.0,13000.0,13500.0,14000.0,14200.0,14400.0,14400.0,14400.0,14600.0,13.4k,4.5709,-74.2973
8,"Congo, Dem. Rep.",842,870,901,945,999,1030,1020,1020,1050,1060,1040,-4.0383,21.7587
9,"Congo, Rep.",4410,4370,4670,4530,4710,4620,4120,3800,3620,3580,3280,-0.2280,15.8277


In [18]:
def clean_and_convert(value):
    if isinstance(value, str) and 'k' in value:
        try:
            numeric_part = float(value.replace('k', ''))
            return numeric_part * 1000
        except ValueError:
            return value  # Return original value if conversion fails
    return value

for col in df.columns[0:11]:  # Apply to columns 1 to 11 (index 0 to 10)
    df[col] = df[col].apply(clean_and_convert)

df

<ipython-input-18-5311102085ee>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(clean_and_convert)


,country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Latitude,Longitude
0,Algeria,11000.0,11100.0,11200.0,11200.0,11400.0,11600.0,11800.0,11700.0,11700.0,11600.0,10.8k,28.0339,1.6596
1,Angola,7690,7660,8010,8100,8180,7970,7490,7220,6880,6600,6030,-11.2027,17.8739
2,Bangladesh,3400,3570,3760,3930,4120,4340,4590,4830,5120,5470,5590,24.0000,90.0000
3,Benin,2630,2630,2680,2790,2880,2850,2860,2930,3040,3160,3190,9.3077,2.3158
4,Brazil,14900.0,15300.0,15500.0,15800.0,15700.0,15000.0,14400.0,14500.0,14700.0,14700.0,14.1k,-14.2350,-51.9253
5,Cameroon,3310,3320,3380,3450,3540,3630,3680,3700,3740,3770,3680,7.3697,12.3547
6,China,8880,9680,10400.0,11100.0,11900.0,12600.0,13400.0,14200.0,15100.0,16000.0,16.3k,35.8617,104.1954
7,Colombia,11900.0,12600.0,13000.0,13500.0,14000.0,14200.0,14400.0,14400.0,14400.0,14600.0,13.4k,4.5709,-74.2973
8,"Congo, Dem. Rep.",842,870,901,945,999,1030,1020,1020,1050,1060,1040,-4.0383,21.7587
9,"Congo, Rep.",4410,4370,4670,4530,4710,4620,4120,3800,3620,3580,3280,-0.2280,15.8277


In [19]:
# prompt: convert values in columns 1:11  to numeric

# Assuming df is your DataFrame and the code to read and process it is already executed.
# The provided code already includes the conversion logic.  If you need additional
# conversions or error handling, please provide specific requirements.

# Example of converting a specific column to numeric (if not already done):
# Assuming the 'GDP per capita' column needs further conversion:

def convert_to_numeric(value):
    try:
        return pd.to_numeric(value, errors='coerce') # errors='coerce' handles non-numeric values gracefully
    except:
        return value

for col in df.columns[1:11]:
    df[col] = df[col].apply(convert_to_numeric)

df

<ipython-input-19-59cb5580af95>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(convert_to_numeric)


,country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Latitude,Longitude
0,Algeria,11000.0,11100.0,11200.0,11200.0,11400.0,11600.0,11800.0,11700.0,11700.0,11600.0,10.8k,28.0339,1.6596
1,Angola,7690.0,7660.0,8010.0,8100.0,8180.0,7970.0,7490.0,7220.0,6880.0,6600.0,6030,-11.2027,17.8739
2,Bangladesh,3400.0,3570.0,3760.0,3930.0,4120.0,4340.0,4590.0,4830.0,5120.0,5470.0,5590,24.0000,90.0000
3,Benin,2630.0,2630.0,2680.0,2790.0,2880.0,2850.0,2860.0,2930.0,3040.0,3160.0,3190,9.3077,2.3158
4,Brazil,14900.0,15300.0,15500.0,15800.0,15700.0,15000.0,14400.0,14500.0,14700.0,14700.0,14.1k,-14.2350,-51.9253
5,Cameroon,3310.0,3320.0,3380.0,3450.0,3540.0,3630.0,3680.0,3700.0,3740.0,3770.0,3680,7.3697,12.3547
6,China,8880.0,9680.0,10400.0,11100.0,11900.0,12600.0,13400.0,14200.0,15100.0,16000.0,16.3k,35.8617,104.1954
7,Colombia,11900.0,12600.0,13000.0,13500.0,14000.0,14200.0,14400.0,14400.0,14400.0,14600.0,13.4k,4.5709,-74.2973
8,"Congo, Dem. Rep.",842.0,870.0,901.0,945.0,999.0,1030.0,1020.0,1020.0,1050.0,1060.0,1040,-4.0383,21.7587
9,"Congo, Rep.",4410.0,4370.0,4670.0,4530.0,4710.0,4620.0,4120.0,3800.0,3620.0,3580.0,3280,-0.2280,15.8277


In [21]:
df['mean'] = df.iloc[:, 1:11].mean(axis=1)

<ipython-input-21-cb30045dd646>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df.iloc[:, 1:11].mean(axis=1)


In [31]:
from ipyleaflet import Map, CircleMarker

# Calculate center using grouped data
center = (df['Latitude'].mean(), df['Longitude'].mean())

m = Map(center=center, zoom=2)

for index, row in df.iterrows():
    marker = CircleMarker(
        location=(row['Latitude'], row['Longitude']),
        radius=int(row['mean'] / 1700),  # Convert to int
        color='purple',
        fill_color='purple',
        fill_opacity=0.6
    )
    m.add_layer(marker)

display(m)
m.save('my_map.html', title='My Map')

Map(center=[8.130765306122449, 15.426828571428572], controls=(ZoomControl(options=['position', 'zoom_in_text',…